
# Generate surface currents

Load all necessary modules

In [ ]:
using CoastalCurrents
using CoastalCurrents: Altimetry
using DIVAnd_HFRadar
using PhysOcean
using GeoMapping
using PyPlot
using NCDatasets
using Dates
using Test
using DIVAnd
using DataStructures

In [ ]:
include("common.jl")

Load the data generated by prep_altimetry notebook/script
`mdt`: mean dynamic topography
`sla`: sea-level anomalies
`lon`,`lat`,`time`: longitude,latitude and time

In [ ]:
ds = NCDataset(altimetry_fname)
mdt = NCDatasets.loadragged(ds["mdt"],:)
sla = NCDatasets.loadragged(ds["slaf"],:)
lon = NCDatasets.loadragged(ds["lon"],:)
lat = NCDatasets.loadragged(ds["lat"],:)
time = NCDatasets.loadragged(ds["time"],:)
id = NCDatasets.loadragged(ds["id"],:)
close(ds)

Compute absolute mean dynamic topography (`adt`)

In [ ]:
adt = mdt .+ sla;

Compute geostrophic velocity

In [ ]:
(lona_,lata_,timea_,ua_,va_) = Altimetry.geostrophic_velocity(lon,lat,time,adt)

Plot a couple of tracks to verify the orientation of the velocity

In [ ]:
clf()
scale = 5
for i = [2 4 6 8]
    scatter(lon[i],lat[i],10,adt[i]);
    r = 1:3:length(lona_[i])
    quiver(lona_[i][r],lata_[i][r],ua_[i][r],va_[i][r],scale=scale,lw = 0.1)
end
colorbar()
CoastalCurrents.Plotting.plotmap(bathname)
title("Geostrophic surface currents")

Concatenate the data into a single vector

In [ ]:
lona = reduce(vcat,lona_)
lata = reduce(vcat,lata_)
timea = reduce(vcat,timea_)
ua = reduce(vcat,ua_)
va = reduce(vcat,va_)

Select the data to be used

In [ ]:
#sel = @. Dates.Month(timea) == 1
sel = @. Dates.DateTime(2000,1,1) <= timea < Dates.DateTime(2001,1,1)
lona = lona[sel]
lata = lata[sel]
timea = timea[sel]
ua = ua[sel]
va = va[sel]

Compute current speed `robs` and direction `directionobs`.
`directionobs` is the angle in degree relative to North counted clock-wise
see DIVAndrun_HFRadar

In [ ]:
robs = sqrt.(ua.^2 + va.^2)
directionobs = atand.(ua,va)

Set-up the domain

In [ ]:
mask,(pm,pn),(xi,yi) = DIVAnd.domain(bathname,bathisglobal,lonr,latr)
mask = DIVAnd.floodfill(mask) .== 1
hx, hy, h = DIVAnd.load_bath(bathname, bathisglobal, lonr, latr)

Ignore data which are NaN

In [ ]:
valid = isfinite.(robs)
x = lona[valid]
y = lata[valid]
robs = robs[valid]
directionobs = directionobs[valid]

Choose analysis parameters

In [ ]:
len = 150e3
epsilon2 = 2.
eps2_boundary_constraint = -1
eps2_div_constraint = 1
g = 0;

Make the DIVAnd analysis

In [ ]:
uri,vri,ηi = DIVAndrun_HFRadar(
    mask,h,(pm,pn),(xi,yi),(lona,lata),robs,directionobs,len,epsilon2;
    eps2_boundary_constraint = eps2_boundary_constraint,
    eps2_div_constraint = eps2_div_constraint,
    # eps2_Coriolis_constraint = -1,
    # f = 0.001,
    # residual = residual,
    # g = g,
    # ratio = 100,
    # lenη = (000.0, 000.0, 24 * 60 * 60. * 10),
    # maxit = 100000,
    # tol = 1e-6,
)

Save the results

In [ ]:
# TODO
timei = timea[1:1]

In [ ]:
CoastalCurrents.save(
    result_filename,
    (xi[:,1],yi[1,:],timei),
    (uri[:,:,1:1],vri[:,:,1:1]))

Plot results

In [ ]:
color = sqrt.(uri.^2 + vri.^2)
clf()
r = CartesianIndices(( 1:2:size(mask,1) ,1:2:size(mask,2)))
r = CartesianIndices(( 1:1:size(mask,1) ,1:1:size(mask,2)))
quiver(xi[r],yi[r],uri[r],vri[r],color[r],cmap="jet")
colorbar(orientation="horizontal")
CoastalCurrents.Plotting.plotmap(bathname)
title("surface current " * join(Dates.format.((minimum(timea),maximum(timea)),"yyyy-mm-dd")," - "))

Visalize with the leaflet javascript library 🍃

In [ ]:
CoastalCurrents.Plotting.plot(xi,yi,uri,vri,scale = 15,maxvelocity = 0.05);